In [4]:
import seisbench
import seisbench.data as sbd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import tensorflow as tf
from tensorflow import keras

In [5]:
data = sbd.LenDB()
print(data)

LenDB - 1244942 traces


In [6]:
rows_earthquake = data.metadata[data.metadata["trace_category"] == "earthquake"]
print(len(rows_earthquake.index))

629095


In [7]:
rows_noise = data.metadata[data.metadata["trace_category"] == "noise"]
print(len(rows_noise.index))

615847


In [8]:
size = 100000
rnd_earthquake = np.random.choice(rows_earthquake.index, size, replace=False)
rnd_noise = np.random.choice(rows_noise.index, size, replace=False)

In [9]:
dataset_e = data.get_waveforms(rnd_earthquake)
print("Earthquake dataset shape:", dataset_e.shape)
label_e = np.ones(size)
#print(label_e)

Earthquake dataset shape: (100000, 3, 540)


In [10]:
dataset_n = data.get_waveforms(rnd_noise)
print("Noise dataset shape:", dataset_n.shape)
label_n = np.zeros(size)
#print(label_e)

Noise dataset shape: (100000, 3, 540)


In [ ]:
mtx = np.concatenate((dataset_e, dataset_n), axis=0)
Y = np.concatenate((label_e, label_n), axis=0)
# Shuffling
random_indexes = np.arange(Y.shape[0])
np.random.shuffle(random_indexes)
mtx = mtx[random_indexes]
Y = Y[random_indexes]

print("data.shape:", mtx.shape)
print("label.shape:", Y.shape)

In [ ]:
# overall_mean = np.mean(X, axis=0, keepdims=True)
# print(overall_mean.shape)
# Xnorm = X - overall_mean
# print(X.shape)
# # print(Xnorm.min())
# # print(Xnorm.max())
# # print(Xnorm)
# ov_std = X.std(axis=0, keepdims=True)
# print(ov_std.shape)
# X = Xnorm/ov_std
# print(X.shape)

In [ ]:
Xnorm = (mtx - mtx.mean(axis =0, keepdims=True) )/ (mtx.std(axis=0, keepdims=True))
# print(X.shape)
# print(Xnorm)
# print(Xnorm.min())
# print(Xnorm.max())
print(Xnorm.std(axis=0))

In [ ]:
# Keras want the channel last so we have to tranpose on the dimension that we want 
X = np.transpose(Xnorm, axes = (0,2,1))
print(X.std(axis=0))
print(X.shape)

In [ ]:
from sklearn.model_selection import train_test_split
X_train_temp, X_temp, y_train_temp, y_temp = train_test_split(X , Y, test_size=0.2)
X_val, X_test, y_val, y_test = train_test_split(X_temp , y_temp, test_size=0.2)

print("Train:", X_train_temp.shape)
#print("Train labels", y_train_temp.mean())
print("Label Train:", y_train_temp.shape)
print("Valid:", X_val.shape)
print("Label valid:", y_val.shape)
print("Test:", X_test.shape)
print("Label Test:", y_test.shape)

In [ ]:
model = tf.keras.models.Sequential([
    
    # Convolution 1
    keras.layers.Conv1D(filters=64, kernel_size=3, strides=2, activation="relu", input_shape=(540,3)),
    # output( 64, 269)
    keras.layers.BatchNormalization(),
    
    # Convolution 2
    keras.layers.Conv1D(filters=64, kernel_size=3, strides=2, activation="relu"),
    # output( 64, 134)
    keras.layers.BatchNormalization(),
    
    # Convolution 3
    keras.layers.Conv1D(filters=64, kernel_size=3, strides=2, activation="relu"),
    # output( 64, 66)
    keras.layers.BatchNormalization(),
    
    # Convolution 4
    keras.layers.Conv1D(filters=64, kernel_size=3, strides=2, activation="relu"),
    # output( 64, 32)
    keras.layers.BatchNormalization(),
    
    # Convolution 5
    keras.layers.Conv1D(filters=64, kernel_size=3, strides=2, activation="relu"),

    keras.layers.Flatten(),
    keras.layers.Dense(1, activation="relu"),
    # keras.layers.Dropout(0.5),
    # keras.layers.Dense(5, activation="softmax")
])

In [ ]:
X_train_temp.std(axis=0)
print(np.abs(X_train_temp).min())
print(np.abs(X_train_temp).max())


In [ ]:
# model.build((None, n_timesteps, n_features, n_outputs))

model.build()
# Check that your model looks right
model.summary()

In [ ]:
from keras.losses import BinaryCrossentropy
# from keras.optimizers import Adam

# adam_opt = Adam(learning_rate=1e-5)
logits_loss = BinaryCrossentropy(
     from_logits=True)

model.compile(loss=logits_loss, optimizer=adam_opt, metrics=['accuracy'])


In [ ]:
model.fit(X_train_temp, y_train_temp, epochs=100, batch_size=64, verbose=1)

In [ ]:
accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)

I need a baseline for the final project